In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
dataset = pd.read_csv('train_data2.csv')
shuffled = dataset.sample(frac=1, random_state=13)
shuffled.head(5)


,resistance_thumb,bend_thumb,resistance_index,bend_index,resistance_mid,bend_mid,resistance_ring,bend_ring,x,y,z,stage
495,16549.50,39,20769.38,64,11311.04,7,17437.89,44,81,37,-150,4
156,10219.69,1,10560.00,3,12654.23,15,7172.00,-16,62,28,-103,0
560,9900.00,0,10923.00,5,9599.12,-2,8109.75,-11,79,38,-155,3
329,25638.46,93,25638.46,93,30360.00,122,13736.25,22,68,34,-157,3
39,10560.00,3,16549.50,39,9900.00,0,9599.12,-2,57,27,-105,1


In [12]:
predictors = shuffled.iloc[:, :11]
predictors[:5]

,resistance_thumb,bend_thumb,resistance_index,bend_index,resistance_mid,bend_mid,resistance_ring,bend_ring,x,y,z
495,16549.50,39,20769.38,64,11311.04,7,17437.89,44,81,37,-150
156,10219.69,1,10560.00,3,12654.23,15,7172.00,-16,62,28,-103
560,9900.00,0,10923.00,5,9599.12,-2,8109.75,-11,79,38,-155
329,25638.46,93,25638.46,93,30360.00,122,13736.25,22,68,34,-157
39,10560.00,3,16549.50,39,9900.00,0,9599.12,-2,57,27,-105


In [13]:
targets = shuffled.iloc[:, -1]
targets[:5]


495    4
156    0
560    3
329    3
39     1
Name: stage, dtype: int64

In [ ]:
SCALE_TOTAL_SALES = 1600.0
trainsize = shuffled,testsize = 0.8
                testsize= len(shuffled['CUST_CODE']) — trainsize
                npredictors = len(predictors.columns)
                noutputs = 1
                
                # change to INFO to get output every 100 steps …
                tf.logging.set_verbosity(tf.logging.WARN)
                print "starting to train … this will take a while … use verbosity=INFO to get more verbose output"
                
                # so that we don’t load weights from previous runs
                shutil.rmtree('./trained_model_linear', ignore_errors=True)
                
                #Instantiate Linear Regression Model
                estimator = tf.contrib.learn.LinearRegressor(model_dir='./trained_model_linear',
                                                            feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values))
                #Train
                def input_fn(features, targets):
                    return tf.constant(features.values), tf.constant(targets.values.reshape(len(targets), noutputs)/SCALE_TOTAL_SALES)
                estimator.fit(input_fn=lambda: input_fn(
                    predictors[:trainsize], targets[:trainsize]), steps=10000)
                pred=np.multiply(list(estimator.predict(
                    predictors[trainsize:].values)), SCALE_TOTAL_SALES)
                rmse_train=np.sqrt(np.mean(np.power((targets[:trainsize].values — pred), 2)))
                rmse_test=np.sqrt(np.mean(np.power((targets[trainsize:].values — pred), 2)))
                print 'LinearRegression — Train has RMSE of {0}'.format(rmse_train)
                print 'LinearRegression — Test has RMSE of {0}'.format(rmse_test)
